In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
import sklearn.preprocessing as skp
import sklearn.metrics.pairwise as smp
from sklearn.cluster import KMeans
import sklearn.decomposition as skd

In [2]:
import pandas as pd

class ProcessAmazon():
    """Read, filter, aggregate by ProductId, and generate X matrix"""

    def __init__(self, raw_data_filename, num_reviews_filter):
        """Initialize attributes."""
        self.raw_data_filename = raw_data_filename
        self.num_reviews_filter = num_reviews_filter
        self.raw_data = self.read_and_report()
        self.grouped_by_userid = self.group_by_userid()
        self.filtered = self.filter_()
        self.grouped_by_productid = self.group_by_productid()
        self.X = self.write_X()
    
    def read_and_report(self):
        """Read Amazon csv, report on file structure"""
        data = pd.read_csv(self.raw_data_filename)
        print(self.raw_data_filename, "has", data.shape[0], "rows and", data.shape[1], "columns. First five observations:")
        print(data.head(5))
        print(50 * '-')
        return data
    
    def group_by_userid(self):
        """Group by UserId and look at the distribution of number of reviews"""
        grouped_by_uid = self.raw_data.groupby('UserId').count()
        print('Distribution of number of reviews for each UserId:')
        print(grouped_by_uid['ProductId'].value_counts())
        print(50 * '-')
        return grouped_by_uid
    
    def filter_(self):
        """Filter out reviewers with fewer than x reviews"""
        filtered = self.grouped_by_userid[self.grouped_by_userid['Id'] > self.num_reviews_filter]
        # remove redundant columns
        filtered_final = filtered.drop(filtered.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1)
        # recreate UserId from index
        filtered_final['UserId'] = filtered_final.index
        # rename columns
        filtered_final.columns = ['num_reviews', 'UserId']
        # left merge filtered list with raw data
        filtered = pd.merge(filtered_final, self.raw_data, how='left', on='UserId')
        print("Filtered data has", filtered.shape[0], "rows and", filtered.shape[1], "columns. First five observations:")
        print(filtered.head(5))
        print(50 * '-')
        return filtered
    
    def group_by_productid(self):
        """Group by ProductId and create a column that lists each UserId (separated by spaces) who reviewed that product"""
        grouped_by_pid = self.filtered.groupby('ProductId')['UserId'].apply(lambda x: ' '.join(x)).reset_index()
        print("Data grouped by ProductId has", grouped_by_pid.shape[0], "rows and", grouped_by_pid.shape[1], "columns. First five observations:")
        print(grouped_by_pid.head(5))
        print(50 * '-')
        return grouped_by_pid
    
    def write_X(self):
        """use bag of words to create boolean X to represent reviews by UserId (one UserId per column)"""
        from sklearn.feature_extraction.text import CountVectorizer
        count = CountVectorizer()
        uidArray = self.grouped_by_productid.UserId.as_matrix()
        userFeatures = count.fit_transform(uidArray)
        print("Shape of final X matrix:", userFeatures.shape)
        return userFeatures

In [3]:
import pandas as pd
class ViewClusteringResults():
    """Merge cluster definitions to raw data and view samples of clustering results, showing review summaries and texts."""
    
    def __init__(self, data_instance, model_instance):
        """Initialize attributes"""
        self.data_instance = data_instance
        self.model_instance = model_instance
    
    def merge_labels(self):
        """Merge cluster labels to raw data (as structured in Amazon.csv)"""
        labels_tomerge = pd.DataFrame(self.model_instance.labels_)
        labelled_products = pd.concat([self.data_instance.grouped_by_productid, labels_tomerge], axis=1, ignore_index=True)
        labelled_products.columns = ['ProductId', 'UserIds', 'cluster']
        merged = pd.merge(labelled_products, self.data_instance.raw_data, how='left', on='ProductId')
        merged_lite = merged[['ProductId', 'cluster', 'Summary', 'Text']]
        return merged_lite

    def view_cluster_samples(self, n_samples):
        """Print x samples of cluster results for each cluster, showing details of product reviews."""
        merged = self.merge_labels()
        num_clusters = self.model_instance.cluster_centers_.shape[0]
        for i in range(1, num_clusters): 
            print(50 * '-')
            print('cluster: ', i)
            cluster = merged.loc[merged['cluster'] == i]
            print('cluster size: ', cluster.shape)
            print(cluster.sample(n_samples))

In [4]:
# process Amazon.csv, filtering out people who left 4 or fewer reviews
amazonData = ProcessAmazon('/Users/michelleponce/Dropbox/Amazon.csv', 5)

/Users/michelleponce/Dropbox/Amazon.csv has 455000 rows and 13 columns. First five observations:
   Unnamed: 0      Id   ProductId          UserId       ProfileName  \
0      138806  138807  B000E63LME  A1CQGW1AOD0LF2  Alena K. "Alena"   
1      469680  469681  B004ZIH4KM  A37S7U1OX2MCWI        Becky Cole   
2      238202  238203  B003ZXE9QA  A2OM6G73E64EQ9              jeff   
3      485307  485308  B001RVFERK  A25W349EE97NBK          Tangent4   
4      375283  375284  B000OQZNTS  A3CPPW0HUC07YS       Amy Nicolai   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       2      2  1294185600   
1                     0                       0      5  1349740800   
2                     0                       0      5  1329264000   
3                     1                       1      4  1248307200   
4                     0                       0      5  1333238400   

                     Summary  \
0           Not as pict

In [8]:
TSVD = skd.TruncatedSVD(n_components=1500, n_iter=50)
new_amazonData=TSVD.fit_transform(amazonData.X)

KeyboardInterrupt: 

In [6]:
print(new_amazonData.dtype)
print(new_amazonData.shape)

float64
(28199, 100)


In [7]:
VR = TSVD.explained_variance_ratio_
print(VR)
print(sum(VR))

[ 0.02175334  0.01935394  0.01729037  0.01661041  0.01541341  0.01321621
  0.01258502  0.01253014  0.0122663   0.01062641  0.0090641   0.00899038
  0.00789702  0.0078378   0.00755747  0.00740908  0.00665184  0.00636059
  0.00605009  0.00576527  0.00572351  0.0054217   0.00520227  0.00511168
  0.00489563  0.00474747  0.00474526  0.00466367  0.00441913  0.00433748
  0.00428544  0.00426579  0.00419295  0.00397172  0.00387393  0.00382663
  0.00375226  0.00373941  0.0036304   0.00356337  0.00352453  0.00339392
  0.00334188  0.00332168  0.00324564  0.00320534  0.0031936   0.00312405
  0.00305618  0.00295868  0.00285166  0.00285266  0.00270517  0.00268602
  0.0025979   0.00248575  0.00238923  0.00234494  0.00234087  0.00231307
  0.00230281  0.00218551  0.00215433  0.0021311   0.00210104  0.00208283
  0.0020492   0.00203922  0.00196759  0.0018785   0.00184193  0.00181057
  0.00181047  0.0017993   0.00171376  0.00170124  0.0016883   0.00168871
  0.00165805  0.00163717  0.00162843  0.0016136   0

In [9]:
km = KMeans(n_clusters=15, max_iter=500, n_init=15, random_state=1)
km.fit(amazonData.X)
#go look at max-iter and be more generous

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=500,
    n_clusters=15, n_init=15, n_jobs=1, precompute_distances='auto',
    random_state=1, tol=0.0001, verbose=0)

In [10]:
# create an instance of ViewClusteringResults to view results of final KMeans fit
viz = ViewClusteringResults(amazonData, km)

In [12]:
# view samples of product reviews, organized by KMeans cluster results
viz.view_cluster_samples(15)

--------------------------------------------------
cluster:  1
cluster size:  (4097, 4)
         ProductId  cluster                                        Summary  \
130166  B0013NUGDE        1                                         Yummo!   
201064  B001RVFEP2        1        My husband takes them to work every day   
351396  B007M83302        1                        Packaging is misleading   
351630  B007M83302        1                                    i'm hooked!   
351400  B007M83302        1                                        Robsmom   
113671  B000VK8AVK        1                               These are yummy!   
201288  B001RVFERK        1                                          Tasty   
200770  B001RVFEP2        1  THESE HAVE CHANGED....AND NOT FOR THE BETTER!   
201411  B001RVFERK        1                                        Awesome   
351639  B007M83302        1                                  I loved them!   
340250  B006HYLW32        1            The Salt & Pepp

In [8]:
# print(amazonData.X.dtype)
# print(amazonData.X.shape)

In [13]:
labels = km.labels_ 
centroids = km.cluster_centers_

print(labels)
print(centroids)


[0 0 0 ..., 0 0 0]
[[  2.50725313e-04   2.50725313e-04   6.80540134e-04 ...,   5.37268527e-04
    2.14907411e-04   2.14907411e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   8.18181818e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [13]:
amazonData_new = km.transform(amazonData.X)
print(amazonData_new)

[[  1.41493889  19.9814729    6.09644705 ...,  18.91305643  13.87637269
   11.63977556]
 [ 11.87017764  23.20041507  13.22245565 ...,  22.28685047  18.20861661
   16.56756998]
 [ 13.33702858  23.98456294  14.5544953  ...,  23.10202813  19.19775297
   17.64891994]
 ..., 
 [  1.99983247  20.03145674   6.25832779 ...,  18.96585626  13.94825147
   11.72537313]
 [  7.74321787  21.30032168   9.75534042 ...,  20.26856716  15.78287596
   13.9098661 ]
 [  2.99966145  20.15587406   6.64580068 ...,  19.09721717  14.1263484
   11.93668191]]
